**Representation**

I represented each district (0 to 6) with a set of 2 qubits. The value of the qubits represented the konbini chains $A:|00\rangle, B:|01\rangle, C:|10\rangle, D:|11\rangle$. A superposition $ 1 = \frac{|00\rangle + |01\rangle}{\sqrt 2}$ means that district 1 (represented by qubits 2 and 3) is equaly lightly to have a kobini store from either chain A or B.

Since there are 7 districts of interest we use 14 qubits to represent them. We use Grover search so we need another qubit to store the result of the oracle. Since the simulator has 32 qubits we are left with 17 qubits to use as ancillas.

**Initialization**

By looking at the initial restrictions commming from districts A, B, C and D it's clear that the initial state of the system can be set to a space that is a fraction of the maximum superposition space. I will then use the general form of Gover's Search also known as Amplitude Amplification to get the right answers.
<img src="./tokyo_map_colored.png" width="700">

The initial superpositions are (I am omitting the normalization values):
$$0 = |01> + |10> + |11>$$
$$1 = |00\rangle + |10\rangle + |11\rangle$$
$$2 = |01\rangle + |11\rangle$$
$$3 = |01\rangle + |10\rangle + |11\rangle$$
$$4 = |00\rangle + |10\rangle + |11\rangle$$
$$5 = |00\rangle + |01\rangle + |10\rangle$$
$$6 = |00\rangle + |01\rangle + |10\rangle$$
After a little thought you can see that these initial superpositions can be created using the methods below `set123()` to `set13()`. The method `A()` then incorporates these methods to set the whole initial state. The method `ADag()` is the adjoint operation.

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute

def set123(a0, a1): 
    qc.x(q[a1])
    qc.h(q[a0])
    qc.cx(q[a0], q[a1])
    qc.ch(q[a0], q[a1])

def set023(a0, a1): 
    qc.h(q[a0])
    qc.ch(q[a0], q[a1])

def set012(a0, a1): 
    qc.h(q[a1])
    qc.x(q[a1])
    qc.ch(q[a1], q[a0])
    qc.x(q[a1])

def set13(a0, a1): 
    qc.h(q[a0])
    qc.x(q[a1])

def set123Dag(a0, a1):
    qc.ch(q[a0], q[a1])
    qc.cx(q[a0], q[a1])
    qc.h(q[a0])
    qc.x(q[a1])

def set023Dag(a0, a1):
    qc.ch(q[a0], q[a1])
    qc.h(q[a0])

def set012Dag(a0, a1): 
    qc.x(q[a1])
    qc.ch(q[a1], q[a0])
    qc.x(q[a1])
    qc.h(q[a1])

def A():
    set123(0, 1)
    set023(2, 3)
    set13(4, 5)
    set123(6, 7)
    set023(8, 9)
    set012(10, 11)
    set012(12, 13)

def ADag():
    set012Dag(12, 13)
    set012Dag(10, 11)
    set023Dag(8, 9)
    set123Dag(6, 7)
    set13(4, 5)
    set023Dag(2, 3)
    set123Dag(0, 1)

**The Oracle**

The idea of the orcle is to compare for equality the states of the districts (pair of qubits) that are linked by an edge in the graph. The oracle should mark as valid those superpositions in which all compared states are different.

As stated above we have 17 qubits that we can use as ancillas. There are 13 edges between the 0..6 districts. We store the results of the 13 comparisons in 13 of the ancilla qubits. In order to make sure all comparisons resulted in the states being different we need a Multiple Controlled Toffoli (mct) gate to mark the state only if all of the 3 qubits that store the comparison results are in the $|0\rangle$ state. The mct gate need a number of ancilla qubits that is equal to `number_of_controll_qubits` - 2. In our case 13-2 = 11. But we already used 13 of our 17 ancillas and we only have 4 left. To get arround this we use the mct gate in the `mode='basic-dirty-ancilla'`. This means that the gate can use ancillas that are not in the 0 state. The gate, after doing its job, leaves the states of the ancillas as they were so we can use our main qubits (used for storing the districts) as the dirty ancillas.

*Optimisations*
There are a number of ways of checking the equality of qubit states but many are complicated and some use ripple carry method or subtraction. As we don't need a general comparer we can create our own custom one that only works for 2 on 2 qubits. This is done in the `equalsNoCcx()` method. The method compares the states in qubits a0 and a1 to b0 and b1 and leaves the result in qubits b0 and b1 (if both are $|1\rangle$ then the states are equal)

Tipically each comparison must be done 4 times: One to get the result and store it in an ancilla, second reverse the comparison in order to restore the qubits for another comparison, then, after the oracle marked the state we need to repeat the process to clear the ancilla. For each of the 7 disctict we can avoid doing 2 comparisons: when doing the last comparison for that district do not restore the state - this way, after the oracle marked the entire superposition of the system, our district qubits are in the correct state for clearing the ancilla.

Since we use our main qubits as the ancillas for the mct gate we are left with 4 unused qubits. We can use those to reduce the number of controll qubits for the mct by storing intermediate results in them.



In [2]:
def equalsNoCcx(a0, a1, b0, b1): 
    qc.cx(q[a0], q[b0])
    qc.x(q[b0])
    qc.cx(q[a1], q[b1])
    qc.x(q[b1])

def equalsNoCcxDag(a0, a1, b0, b1): 
    qc.x(q[b1])
    qc.cx(q[a1], q[b1])
    qc.x(q[b0])
    qc.cx(q[a0], q[b0])
    
def computeEqualities():
    equalsNoCcx(2,3,6,7)
    qc.rccx(q[6], q[7], q[14])
    equalsNoCcxDag(2,3,6,7)
    
    equalsNoCcx(2,3,8,9)
    qc.rccx(q[8], q[9], q[15])
    equalsNoCcxDag(2,3,8,9)
    
    equalsNoCcx(4,5,6,7)
    qc.rccx(q[6], q[7], q[16])
    equalsNoCcxDag(4,5,6,7)
    
    equalsNoCcx(4,5,10,11)
    qc.rccx(q[10], q[11], q[17])
    equalsNoCcxDag(4,5,10,11)
    
    equalsNoCcx(4,5,12,13)
    qc.rccx(q[12], q[13], q[18])
    equalsNoCcxDag(4,5,12,13)
    
    equalsNoCcx(6,7,12,13)
    qc.rccx(q[12], q[13], q[19])
    equalsNoCcxDag(6,7,12,13)
    
    equalsNoCcx(8,9,12,13)
    qc.rccx(q[12], q[13], q[20])
    equalsNoCcxDag(8,9,12,13)
    
    equalsNoCcx(10,11,12,13)
    qc.rccx(q[12], q[13], q[21])
    
    equalsNoCcx(0,1,2,3)
    qc.rccx(q[2], q[3], q[22])
    
    equalsNoCcx(0,1,4,5)
    qc.rccx(q[4], q[5], q[23])
        
    equalsNoCcx(6,7,8,9)
    qc.rccx(q[8], q[9], q[24])
    
    equalsNoCcx(6,7,10,11)
    qc.rccx(q[10], q[11], q[25])
    
    equalsNoCcx(0,1,6,7)
    qc.rccx(q[6], q[7], q[26])
    
def computeEqualitiesDag():
    qc.rccx(q[6], q[7], q[26])
    equalsNoCcxDag(0,1,6,7)
    
    qc.rccx(q[10], q[11], q[25])
    equalsNoCcxDag(6,7,10,11)
        
    qc.rccx(q[8], q[9], q[24])
    equalsNoCcxDag(6,7,8,9)
    
    qc.rccx(q[4], q[5], q[23])
    equalsNoCcxDag(0,1,4,5)
    
    qc.rccx(q[2], q[3], q[22])
    equalsNoCcxDag(0,1,2,3)
    
    qc.rccx(q[12], q[13], q[21])
    equalsNoCcxDag(10,11,12,13)
    
    equalsNoCcx(8,9,12,13)
    qc.rccx(q[12], q[13], q[20])
    equalsNoCcxDag(8,9,12,13)
    
    equalsNoCcx(6,7,12,13)
    qc.rccx(q[12], q[13], q[19])
    equalsNoCcxDag(6,7,12,13)
    
    equalsNoCcx(4,5,12,13)
    qc.rccx(q[12], q[13], q[18])
    equalsNoCcxDag(4,5,12,13)
    
    equalsNoCcx(4,5,10,11)
    qc.rccx(q[10], q[11], q[17])
    equalsNoCcxDag(4,5,10,11)
    
    equalsNoCcx(4,5,6,7)
    qc.rccx(q[6], q[7], q[16])
    equalsNoCcxDag(4,5,6,7)
    
    equalsNoCcx(2,3,8,9)
    qc.rccx(q[8], q[9], q[15])
    equalsNoCcxDag(2,3,8,9)
    
    equalsNoCcx(2,3,6,7)
    qc.rccx(q[6], q[7], q[14])
    equalsNoCcxDag(2,3,6,7)
    
def oracle():
    computeEqualities()
    
    qc.x(q[14:27])
    qc.rccx(q[14], q[15], q[28])
    qc.rccx(q[16], q[17], q[29])
    qc.rccx(q[18], q[19], q[30])
    qc.rccx(q[20], q[21], q[31])
    qc.mct([q[22],q[23],q[24],q[25],q[26],q[28],q[29],q[30],q[31]], q[27] , [q[0], q[1], q[2], q[3], q[4], q[5], q[6]], mode='basic-dirty-ancilla')
    qc.rccx(q[14], q[15], q[28])
    qc.rccx(q[16], q[17], q[29])
    qc.rccx(q[18], q[19], q[30])
    qc.rccx(q[20], q[21], q[31])
    qc.x(q[14:27])
    
    computeEqualitiesDag()

**Phase Shift**

The phase shift operation is tipical: a multiple controlled Z gate sandwitched between X gates. This time we do have enough free ancillas for the mct gate.

In [3]:
def phaseShift():
    qc.x(q[0:14])
    qc.barrier()
    qc.h(q[13])
    qc.mct(q[0:13], q[13] , q[14:25], mode='basic')
    qc.h(q[13])
    qc.barrier()
    qc.x(q[0:14])

**Final Optimizations**

As a final optimization notice that when using toffoli gates we only need them up to a relative phase so we can replace all the `ccx()` gates with `rccx()`

In [4]:
q = QuantumRegister(32)
c = ClassicalRegister(32)
qc = QuantumCircuit(q,c)

qc.x(q[27])
qc.h(q[27])
A()
#qc.barrier()

def groverIteration():
    oracle()
    ADag()
    phaseShift()
    A()

for i in range(0,5):
    groverIteration()

qc.measure(q[0:14], c[0:14])

In [7]:
#provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
job = execute(qc, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count = result.get_counts()
print(count)

c:\users\sorin\anaconda3\envs\qiskit\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:181: UserWarning: Credentials are already in use. The existing account in the session will be replaced.
  warnings.warn('Credentials are already in use. The existing '


{'00000000000000000001101110100011': 1, '00000000000000000010010101100110': 1, '00000000000000000010000110110011': 6, '00000000000000000010011111111111': 2, '00000000000000000000100110110001': 5, '00000000000000000001100001100001': 3, '00000000000000000000011101111111': 2, '00000000000000000010100101110011': 2, '00000000000000000000010111100001': 187, '00000000000000000010011110110101': 7, '00000000000000000001010010100101': 2, '00000000000000000000011111110010': 1, '00000000000000000001101101100001': 1, '00000000000000000000100101100111': 1, '00000000000000000001010110100111': 1, '00000000000000000001101101100101': 3, '00000000000000000010000010110110': 8, '00000000000000000010100001111101': 3, '00000000000000000010100011111110': 11, '00000000000000000000100111100111': 1, '00000000000000000001101101101111': 2, '00000000000000000010010010110111': 3, '00000000000000000010000001100010': 16, '00000000000000000000000111101110': 2, '00000000000000000000000101100010': 5, '0000000000000000000

In [8]:
# Input your quantum circuit
circuit=qc
# Input your result of the execute(groverCircuit, backend=backend, shots=shots).result()
results = result
count=results.get_counts()
# Provide your team name
name='Sorin'
# Please indicate the number of times you have made a submission so far. 
# For example, if it's your 1st time to submit your answer, write 1. If it's your 5th time to submit your answer, write 5.
times='6'

In [9]:
import json
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(circuit) 

# obtain gates
gates=new_circuit.count_ops()

#sort count
count_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)

# collect answers with Top 9 probability
ans_list = count_sorted[0:9]

# reverse ans_list
ans_reversed = []
for i in ans_list:
    ans_temp=[i[0][::-1],i[1]]
    ans_reversed.append(ans_temp)

# convert each 2 bits into corresponding color. Add node0(0),node3(1),node8(2) and node11(3)
ans_shaped = []
for j in ans_reversed:
    ans_temp=j[0]
    nodeA = 0
    node0 = int(ans_temp[0] + ans_temp[1], 2)
    node1 = int(ans_temp[2] + ans_temp[3], 2)
    nodeB = 1
    node2 = int(ans_temp[4] + ans_temp[5], 2)
    node3 = int(ans_temp[6] + ans_temp[7], 2)
    node4 = int(ans_temp[8] + ans_temp[9], 2)
    nodeC = 2
    node5 = int(ans_temp[10] + ans_temp[11], 2)
    node6 = int(ans_temp[12] + ans_temp[13], 2)
    nodeD = 3
    nodes_color = str(nodeA) + str(node0) + str(node1) + str(nodeB) + str(node2) + str(node3) + str(node4) + str(nodeC) + str(node5) + str(node6) + str(nodeD) 
    ans_shaped.append([nodes_color,j[1]])

# write the result into '[your name]_final_output.txt'
filename=name+'_'+times+'_final_output.txt'
dct={'ans':ans_shaped,'costs':gates}
with open(filename, 'w') as f:
    json.dump(dct, f)


In [10]:
# Input the path of your submission file
your_path='./Sorin_6_final_output.txt'

In [11]:
import json
from pathlib import Path

p= Path(your_path)

# Verify your information
f_name=p.name
your_info=f_name.split('_')
print('Your name: ', your_info[0])
print('The number of times you have submitted your answer: ', your_info[1])

with open(p, 'r') as f:
    print(f)
    your_ans=json.load(f)

print('Does your submission file have 9 answers?')
if (len(your_ans['ans'])!=9):
    print('- No, make sure you have 9 answers with top 9 probabilities')
else:
    print('- Yes')
    print('- Your plan: ', your_ans['ans'])

print('What is your cost?')
your_cost=your_ans['costs']['u3'] + 10*your_ans['costs']['cx']
your_cost

Your name:  Sorin
The number of times you have submitted your answer:  6
<_io.TextIOWrapper name='Sorin_6_final_output.txt' mode='r' encoding='cp1252'>
Does your submission file have 9 answers?
- Yes
- Your plan:  [['01313202013', 792], ['01013232103', 754], ['01013232013', 720], ['02013132023', 389], ['02313102023', 389], ['02013132203', 365], ['02013122203', 356], ['02011322203', 187], ['02313122203', 179]]
What is your cost?


21667